# Use inversion framework to fit a linear regressor

In [1]:
import numpy as np

from inversion_ideas import DataMisfit, TikhonovZero, ConjugateGradient
from regressor import LinearRegressor

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X array
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
noise = rng.normal(scale=1e-2 * maxabs, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

In [5]:
simulation = LinearRegressor(X)

In [6]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)
smallness = TikhonovZero(n_params)

In [7]:
phi = data_misfit + 1e-3 * smallness
phi

φ(m) + 0.00 φ(m)

In [8]:
initial_model = np.zeros(n_params)
initial_model

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Minimize manually with `scipy.sparse.linalg.cg`

In [9]:
from scipy.sparse.linalg import cg

In [10]:
grad = phi.gradient(initial_model)
hess = phi.hessian(initial_model)

In [11]:
model_step, info = cg(hess, -grad)
model_step, info

(array([0.81328361, 0.65926983, 0.24729306, 0.19624665, 0.32373823,
        0.14720994, 0.31944798, 0.25236328, 0.52215617, 0.92180399]),
 0)

In [12]:
inverted_model = initial_model + model_step

In [13]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.81328361 0.65926983 0.24729306 0.19624665 0.32373823 0.14720994
 0.31944798 0.25236328 0.52215617 0.92180399]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Minimize with SciPy's `minimize`

In [14]:
from scipy.optimize import minimize

In [15]:
result =  minimize(phi, initial_model)
result

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.37159603111570305
        x: [ 8.133e-01  6.592e-01  2.473e-01  1.963e-01  3.237e-01
             1.472e-01  3.195e-01  2.524e-01  5.222e-01  9.218e-01]
      nit: 16
      jac: [-1.378e-07  2.645e-07  1.155e-07  1.118e-08 -1.006e-07
             1.714e-07 -3.725e-08  2.682e-07  3.353e-08  1.602e-07]
 hess_inv: [[ 1.077e-02 -3.331e-03 ...  5.039e-04 -2.987e-03]
            [-3.331e-03  1.286e-02 ... -3.213e-03 -4.085e-04]
            ...
            [ 5.039e-04 -3.213e-03 ...  6.867e-03 -1.745e-03]
            [-2.987e-03 -4.085e-04 ... -1.745e-03  8.357e-03]]
     nfev: 231
     njev: 21

In [16]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [17]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.81325615 0.65924881 0.24726449 0.19626688 0.32373389 0.14719947
 0.31947497 0.25235411 0.5221697  0.92183533]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [18]:
result =  minimize(phi, initial_model, jac=phi.gradient, method="CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.3715960311156822
       x: [ 8.133e-01  6.592e-01  2.473e-01  1.963e-01  3.237e-01
            1.472e-01  3.195e-01  2.524e-01  5.222e-01  9.218e-01]
     nit: 12
     jac: [ 8.321e-13 -8.337e-13 -1.028e-12 -2.157e-14 -2.755e-13
           -4.928e-13 -8.490e-13 -1.512e-12 -5.860e-13 -1.837e-12]
    nfev: 24
    njev: 24

In [19]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [20]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.81325615 0.65924881 0.2472645  0.19626689 0.32373389 0.14719947
 0.31947497 0.25235411 0.5221697  0.92183533]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [21]:
result =  minimize(phi, initial_model, jac=phi.gradient, hess=phi.hessian, method="Newton-CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.37159603111728207
       x: [ 8.133e-01  6.592e-01  2.473e-01  1.963e-01  3.237e-01
            1.472e-01  3.195e-01  2.524e-01  5.222e-01  9.218e-01]
     nit: 10
     jac: [-1.755e-04 -1.486e-04  1.521e-04  4.685e-05  3.222e-05
           -1.612e-04  8.816e-05  7.859e-06  3.653e-04 -1.935e-04]
    nfev: 11
    njev: 11
    nhev: 28

In [22]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [23]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.81325626 0.65924868 0.24726445 0.19626684 0.32373384 0.14719954
 0.31947496 0.25235426 0.52216973 0.92183526]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Use `Minimizer` class

In [24]:
minimizer = ConjugateGradient()
minimizer

In [25]:
inverted_model = minimizer(phi, initial_model)

In [26]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.81328361 0.65926983 0.24729306 0.19624665 0.32373823 0.14720994
 0.31944798 0.25236328 0.52215617 0.92180399]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]
